<a href="https://colab.research.google.com/github/RohitGottapu/lunar_lander_Rl/blob/main/lunarlanding_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install gymnasium

In [8]:
import gymnasium as gym

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
pip install swig

  Using cached swig-4.3.1.post0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (3.5 kB)
Using cached swig-4.3.1.post0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.9 MB)


In [13]:
pip install "gymnasium[box2d]"

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2409504 sha256=e709f657f0bc0f591fb4327adcc358e762891f947d7d1943af02c1872e2a9511
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


In [14]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import variable
from collections import deque,namedtuple

In [15]:
env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to 

In [16]:
state=env.observation_space.shape
state_size=env.observation_space.shape[0]
action_size=env.action_space.n
print(state,state_size,action_size)

(8,) 8 4


In [36]:
#hyper parameters
learning_rate=5e-4
minibatch=150
gamma=0.99
replay_buffer_size=100000
interpolation_paramenter=1e-3
number_episodes=5000
max_time_steps=1000 #per episode
epsilon_starting_value=1
epsilon_ending_value=0.01
epsilon_decay_value=0.995
score_100_episodes=deque(maxlen=100)

In [35]:
class ANN(nn.Module):
  def __init__(self,state_size,action_size,seed=42):
    super(ANN,self).__init__()
    self.seed=torch.manual_seed(seed)
    self.fc1= nn.Linear(state_size,64)#fc1 is the name given to first layer of NN
    self.fc2= nn.Linear(64,64)
    self.fc3=nn.Linear(64,action_size)

  def forward(self,state):
    x=self.fc1(state)
    x=F.relu(x)
    x=self.fc2(x)
    x=F.relu(x)
    return self.fc3(x)



In [34]:
class ReplayMemory(object):

  def __init__(self,capacity):
    self.capacity=capacity
    self.memory=[]

  def push(self,event):
    self.memory.append(event)
    if len(self.memory)>self.capacity:
      del self.memory[0]
  def sample(self,batch_size):
    experiences= random.sample(self.memory,batch_size)
    #converting to tensors as they should be used for ANN
    states=torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float()
    actions=torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long()
    rewards=torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float()
    next_states=torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float()
    dones=torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float()

    return states,actions,rewards,next_states,dones

  def __len__(self):
      return len(self.memory)

In [33]:
class Agent():
  def __init__(self,state_size,action_size):
    self.state_size=state_size
    self.action_size=action_size
    self.local_qnetwork=ANN(state_size, action_size)
    self.target_qnetwork=ANN(state_size, action_size)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr= learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0
  # stores expericences in the memory and collects minibatch no.of samples from mem and gives them to learn
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if  self.t_step==0:
      if len(self.memory) > minibatch:
        experiences=self.memory.sample(minibatch)
        self.learn(experiences,gamma)
  #given current state gets suitable action from local ANN
  def get_action(self, state, epsilon):
    state= torch.from_numpy(state).float().unsqueeze(0)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random()>epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))
  # updates the ANN's
  def learn(self, experiences, gamma):
    state, action, reward, next_state, done = experiences
    next_q_targets = self.target_qnetwork(next_state).detach().max(1)[0].unsqueeze(1)
    q_targets = reward + (gamma * next_q_targets * (1-done))
    q_expected = self.local_qnetwork(state).gather(1, action)
    loss = F.mse_loss(q_expected,q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_paramenter)
  def soft_update(self, local_qnetwork, target_qnetwork, interpolation_paramenter):
    for target_params, local_params in zip(target_qnetwork.parameters(), local_qnetwork.parameters()):
      target_params.data.copy_(interpolation_paramenter*local_params.data + (1-interpolation_paramenter)*target_params.data)

In [21]:

agent=Agent(state_size, action_size)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [22]:
epsilon = epsilon_starting_value
for episode in range(0, number_episodes):
  state,_ = env.reset()
  score=0
  for st in range(0, max_time_steps):
    #these entire things is a single step
    action = agent.get_action(state,epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score+=reward
    if done:
      break
  score_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon*epsilon_decay_value)
  if episode % 5 == 0 :
    print('Episodes {} avg score {:2f}'.format(episode, np.mean(score_100_episodes)))
  if np.mean(score_100_episodes) >= 200:
    torch.save(agent.local_qnetwork.state_dict(), "lunar_lander_model.pth")
    print('congratulations, solved in {:d} episodes \t avg score:{:2f}'.format(episode, np.mean(score_100_episodes)))

    break


Episodes 0 avg score -66.821175
Episodes 5 avg score -211.726641
Episodes 10 avg score -205.691499
Episodes 15 avg score -192.063384
Episodes 20 avg score -176.000354
Episodes 25 avg score -168.896711
Episodes 30 avg score -169.317118
Episodes 35 avg score -165.987326
Episodes 40 avg score -190.946475
Episodes 45 avg score -185.044526
Episodes 50 avg score -179.864802
Episodes 55 avg score -179.622387
Episodes 60 avg score -174.916334
Episodes 65 avg score -170.801385
Episodes 70 avg score -169.954537
Episodes 75 avg score -163.036996
Episodes 80 avg score -157.369402
Episodes 85 avg score -152.318569
Episodes 90 avg score -152.498975
Episodes 95 avg score -153.388062
Episodes 100 avg score -150.560707
Episodes 105 avg score -144.532348
Episodes 110 avg score -140.381077
Episodes 115 avg score -139.873885
Episodes 120 avg score -139.586157
Episodes 125 avg score -138.508840
Episodes 130 avg score -135.991504
Episodes 135 avg score -138.325308
Episodes 140 avg score -130.094036
Episodes

In [31]:
import glob
import io
import base64
import imageio
import gym
import numpy as np
from IPython.display import HTML

# --- Patch deprecated NumPy attributes for compatibility ---
# (needed if numpy >= 1.24 and gym uses np.bool, np.int, etc.)
if not hasattr(np, "bool"):
    np.bool = bool
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "float"):
    np.float = float
if not hasattr(np, "object"):
    np.object = object
if not hasattr(np, "bool8"):
    np.bool8 = np.bool_

# --- Frame fixer utility ---
def fix_frame(frame):
    frame = np.array(frame, dtype=np.uint8)

    # Remove leading batch dimension if present (1, H, W, C)
    if frame.ndim == 4 and frame.shape[0] == 1:
        frame = frame[0]

    # Convert grayscale → RGB
    if frame.ndim == 2:
        frame = np.stack([frame] * 3, axis=-1)

    return frame


# --- Record agent video ---
def show_video_of_model(agent, env_name="LunarLander-v2"):
    env = gym.make(env_name, render_mode='rgb_array')

    # Handle reset for both gym/gymnasium
    result = env.reset()
    state = result[0] if isinstance(result, tuple) else result

    done = False
    frames = []

    while not done:
        frame = env.render()
        if frame is not None:
            frame = fix_frame(frame)
            frames.append(frame)

        # Agent takes action
        action = agent.get_action(state, 0.0)

        # Always cast action to int to avoid .item() errors
        step_out = env.step(int(action))

        # Handle Gymnasium vs Gym API
        if len(step_out) == 5:  # Gymnasium: (obs, reward, terminated, truncated, info)
            state, reward, terminated, truncated, _ = step_out
            done = terminated or truncated
        else:  # Old Gym: (obs, reward, done, info)
            state, reward, done, _ = step_out

        done = bool(done)  # ensure boolean type

    env.close()

    # Save video
    imageio.mimsave("video.mp4", frames, fps=30)


# --- Display video in Colab ---
def show_video():
    mp4list = glob.glob("*.mp4")
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, "r+b").read()
        encoded = base64.b64encode(video)
        return HTML(f"""
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4">
        </video>
        """)
    else:
        print("Could not find video")


# --- Example usage ---
# (replace `agent` with your trained agent object)
show_video_of_model(agent, "LunarLander-v2")  # use "LunarLander-v3" if gymnasium
show_video()
